In [15]:
import pandas as pd
import numpy as np
import shutil
from pathlib import Path
import re
import random

In [81]:
nums = sorted([int(re.search(r'(\d*)\.jpg', str(p)).group(1)) for p in Path('all').glob('**/*jpg')])
data = pd.read_csv("data.csv", index_col=0)
data.head()

,id,lat,lng,country,population,country_name,geocell
0,ltbLypZTB0b54YldBhcrYQ,61.772846,-149.93478,us,0.101961,United States,10
1,4YLg9of3KX6wKna1hD-Bqw,42.829094,-71.78170,us,0.231373,United States,188
2,VmQWOJCj9lgSQnAbCgH_bA,36.138850,-95.99014,us,0.529412,United States,132
3,FEUW8CKZo3HdyW-O4SJSow,35.028843,-89.81706,us,0.603922,United States,133
4,jW-3dsHVWPvaxA7g-VNW5Q,27.803684,-82.69630,us,0.596078,United States,175


In [53]:
def cutsmall(dt):
    country_counts = dt.country.value_counts().sort_values()
    print("country_counts:", country_counts[:5])
    small_countries = country_counts[country_counts < 50].index.tolist()

    geocell_counts = dt.geocell.value_counts().sort_values()
    print("\ngeocell_counts:", geocell_counts[:5])
    small_geocells = geocell_counts[geocell_counts < 50].index.tolist()

    ret = dt[~ (dt.country.isin(small_countries) | dt.geocell.isin(small_geocells))]
    print(f"\nCut from {len(dt)} to {len(ret)}")
    done = len(dt) == len(ret)
    return ret, done

In [54]:
def cut_rec():
    curdata = data.iloc[nums]
    done = False
    while not done:
        curdata, done = cutsmall(curdata)
    return curdata

In [56]:
data_cut = cut_rec()

country_counts: cx    1
py    1
ml    1
zw    1
ge    1
Name: country, dtype: int64

geocell_counts: 2      5
24     7
1      9
9     11
41    12
Name: geocell, dtype: int64

Cut from 98075 to 97086
country_counts: vn    59
gl    62
pk    63
sz    63
al    64
Name: country, dtype: int64

geocell_counts: 21     18
253    53
137    55
68     56
230    57
Name: geocell, dtype: int64

Cut from 97086 to 97068
country_counts: vn    59
gl    62
pk    63
sz    63
al    64
Name: country, dtype: int64

geocell_counts: 253    53
137    55
68     56
230    57
189    62
Name: geocell, dtype: int64

Cut from 97068 to 97068


In [ ]:
data_cut.to_csv("data.csv")

In [57]:
def make_train_valid_test(idxs):
    random.seed(47)
    random.shuffle(idxs)
    spl_tr_val = int(len(idxs) * 0.8)
    spl_val_tst = int(len(idxs) * 0.9)
    return idxs[:spl_tr_val], idxs[spl_tr_val:spl_val_tst], idxs[spl_val_tst:]

In [60]:
tr, vl, ts = make_train_valid_test(data_cut.index.tolist())

In [61]:
len(tr), len(vl), len(ts)

(77654, 9707, 9707)

In [67]:
def make_data():
    dset_idxs = {'train': tr, 'valid': vl, 'test': ts}
    for dset, idxs in dset_idxs.items():
        for idx in idxs:
            shutil.copy(f"all/{idx}.jpg", f"data/{dset}/{idx}.jpg")

# make_data()